In [1]:
# %load C:\Users\PBS Biotech\Documents\Personal\PBS_Office\MSOffice\hello\dummyhello2.py
"""

Created by: Nathan Starkweather
Created on: 08/01/2014
Created in: PyCharm Community Edition


"""
__author__ = 'Nathan Starkweather'

import threading
from traceback import print_exc
from xml.etree.ElementTree import parse as xml_parse
from urllib.request import urlopen
from urllib.error import URLError
from queue import Queue, Empty
from io import BytesIO
from collections import OrderedDict
import time
import json

from tkinter import Tk, N, StringVar
import tkinter as tk
from tkinter.ttk import LabelFrame, Label

from hello.hello3 import open_hello, HelloApp, TrueError


class _ThreadSyncData():
    def __init__(self):
        
        self.stop_flag = threading.Event()
        
        # data lock
        self.lock = threading.Lock()
        self.data = {}
        
        # heartbeat
        self.hb_good = 0
        self.hb_bad = 0
        
        # timestamps
        self.ts_last_update = time.time()
        self.ts_last_access = time.time()
        
    def set_value(self, v):
        with self.lock:
            self.data = v
            self.ts_last_update = time.time()
            
    def get_value(self):
        with self.lock:
            rv = self.data
            self.ts_last_access = time.time()
            return rv

        
class SimpleLabelFrame(LabelFrame):
    def __init__(self, parent, text, **kw):
        tv = StringVar(None, text)
        self.text_var = tv
        super().__init__(parent, textvariable=tv, **kw)
        
    def set_text(self, t):
        self.tv.set(t)
        
        
class SimpleLabel(Label):
    def __init__(self, parent, text, **kw):
        self.text_var = StringVar(None, text)
        super().__init__(parent, textvariable=self.text_var, **kw)
    
    def set_text(self, t):
        self.text_var.set(t)
    
    def get_text(self, t):
        return self.text_var.get()
        
        
class _CFF():
    def __init__(self, parent, name, value):
        self.l_name = SimpleLabel(parent, name, width=15)
        self.l_value = SimpleLabel(parent, value, width=8)
        
    def set_value(self, v):
        self.l_value.set_text(str(v))
    
    def set_name(self, n):
        self.l_name.set_text(str(n))
        
    def grid(self, row=0, column=0):
        self.l_name.grid(row=row, column=column)
        self.l_value.grid(row=row, column=column+1)
        
        
class ControllerFrame():
    def __init__(self, parent, name):
        self.parent = parent
        self.frame = LabelFrame(parent, text=name, height=50)
        self.fields = OrderedDict()
        
    def add_field(self, key, name, value):
        field = _CFF(self.frame, name, value)
        self.fields[key] = field
        
    def __getitem__(self, key):
        return self.fields[key]
    
    def grid(self, row, column, **kw):
        kw['sticky'] = (tk.N, tk.S)
        self.frame.grid(row=row, column=column, **kw)
        for i, f in enumerate(self.fields.values()):
            f.grid(i, column)
            
    def grid_forget(self):
        self.frame.grid_forget()
        
    
class UpdateThread(threading.Thread):
    def __init__(self, data, update_func, poll_lock, interval=0.1):
        self.data = data
        self.update_func = update_func
        self.interval = interval
        self.poll_lock = poll_lock
        super().__init__(daemon=True)
        
    def run(self):
        sfs = self.data.stop_flag.is_set
        func = self.update_func
        set_value = self.data.set_value
        sleep = time.sleep
        iv = self.interval
        lock = self.poll_lock
        data = self.data 
        def on_err(e):
            print("Error in update thread %s: %s" % (func.__qualname__, e))
        
        # mainloop
        v = None
        while not sfs():
            with lock:
                try:
                    v = func()
                except TrueError:
                    data.hb_bad += 1
                except Exception as e:
                    on_err(e)
                    data.hb_bad += 1
                else:
                    data.hb_good += 1
            set_value(v)
            sleep(iv)
            
    def get_update(self):
        return self.data.get_value()
    
    def stop(self):
        self.data.stop_flag.set()
        
        
# noinspection PyAttributeOutsideInit
class Hello():
    def __init__(self, ipv4):

        self.unack_count = 0
        self.mv_cont = mvc = {}

        self.app = HelloApp(ipv4)
        mv = self.app.send_request("?&call=getMainValues&json=1").read()
        mv = mv.decode()
        mv = json.loads(mv, object_hook=OrderedDict)
        mv2 = OrderedDict()
        for k, v in sorted(mv.items()):
            mv2[k] = OrderedDict(sorted(v.items()))
        self.main_values = mv2
        
        # deep copy
        for k1, v1 in self.main_values.items():
            mvc[k1] = mvcv1 = {}
            for k2, v2 in v1.items():
                mvcv1[k2] = v2
        
        

    def getMainValues(self):
        return self.mv_data.get_value()

    def getUnAckCount(self):
        return self.ua_data.get_value()

    def setup(self):
        
        root = Tk()
        mframe = LabelFrame(root, text="Loading...")
        group_frames = OrderedDict()
        labels = OrderedDict()
        views = OrderedDict()
        max_fields = max(len(v) for v in self.main_values.values())
        for ctrl, names in self.main_values.items():
            frame = ControllerFrame(mframe, ctrl)
            group_frames[ctrl] = frame
            for name, val in names.items():
                frame.add_field(name, name, val)
            to_add = max_fields - len(names)
            for _ in range(to_add):
                frame.add_field("","","")

        for i, frame in enumerate(group_frames.values()):
            frame.grid(sticky=N, column=i, row=0)
            
        for cntlr in labels.values():
            for label in cntlr.values():
                label[0].grid()
                label[1].grid()
                
        mframe.grid()

        self.root = root
        self.mframe = mframe
        self.gframes = group_frames
        self.labels = labels
        self.vars = views

        # Initialize threading model
        poll_lock = threading.Lock()
        self.mv_data = _ThreadSyncData()
        self.mv_thread = UpdateThread(self.mv_data, self.app.gpmv, poll_lock, 0.5)

        self.ua_data = _ThreadSyncData()
        self.ua_data.data = 0
        self.ua_thread = UpdateThread(self.ua_data, self.app.getUnAckCount, poll_lock, 0.5)

        self.mv_thread.start()
        #self.ua_thread.start()

    def mainloop(self):
        self.setup()
        self.root.after(250, self.poll)
        try:
            self.root.mainloop()
        finally:
            self.mv_thread.stop()
            self.ua_thread.stop()
            for t in (self.mv_thread, self.ua_thread):
                t.join()

    def poll(self):

        update = False
        root = self.getMainValues()
        if root:
            self.parse_main_values2(root)
            update = True
            
        unack_count = self.getUnAckCount()
        if unack_count is not None:
            self.unack_count = unack_count

        self.root.after(250, self.update, update)
        self.root.after(500, self.poll)
        
    def format_float(self, f):
        return "%.3f" % float(f)

    def update(self, update_widgets=True):

        if update_widgets:
            for group, names in self.main_values.items():
                frame = self.gframes[group]
                local_data = self.mv_cont[group]
                for name, val in names.items():
                    if local_data[name] != val:
                        local_data[name] = val
                        frame[name].set_value(self.format_float(val))
                    
        self.mframe.configure(text="MV: %d-%d, Alarms: %d" % (self.mv_data.hb_good,
                                                                         self.mv_data.hb_bad,
                                                                         self.unack_count))


    def parse_main_values2(self, dct):
        for group_name, group_dct in dct.items():
            self.main_values[group_name].update(group_dct)

            
def main(ipv4):
    h = Hello(ipv4)
    h.mainloop()
    return 0

if 0:
    if __name__ == '__main__':

        import sys
        try:
            ipv4 = sys.argv[1]
            sys.exit(main(ipv4) or 0)
        except (Exception, KeyboardInterrupt):
            import traceback
            traceback.print_exc()
            input("Error occurred. Press a key to close.")
            sys.exit(1)
        except SystemExit:
            pass



In [ ]:
main('71.189.82.196:81')

In [2]:
h = HelloApp('71.189.82.196:81')